# Predicting model using tensorflow fo stellar clasification

In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/juancv3d/.pyenv/versions/3.8.10/envs/AI/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
2022-07-18 23:13:20.912213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 23:13:20.912290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
stellar = pd.read_csv('./data/star_classification.csv')
stellar

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [27]:
stellar.dtypes

obj_ID         float64
alpha          float64
delta          float64
u              float64
g              float64
r              float64
i              float64
z              float64
run_ID           int64
rerun_ID         int64
cam_col          int64
field_ID         int64
spec_obj_ID    float64
class            int64
redshift       float64
plate            int64
MJD              int64
fiber_ID         int64
dtype: object

In [7]:
stellar['class'].unique()

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [8]:
stellar['class'] = stellar['class'].map({'GALAXY': 0, 'QSO': 1, 'STAR': 2})

In [54]:
stellar['class']

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: class, Length: 100000, dtype: int64

In [10]:
x = stellar.drop(['class'], axis=1)
y = stellar['class']

x_train , x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [12]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

In [31]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.98:
            print("\nReached 98% accuracy so cancelling training!")
            self.model.stop_training = True

In [89]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    
    return model

In [90]:

callbacks = myCallback()
model = build_model()
hist = model.fit(x_train, y_train, epochs=10, callbacks=[myCallback()])



Epoch 1/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.2841 - accuracy: 0.9010
Epoch 2/10
2500/2500 [==============================] - 11s 5ms/step - loss: 0.1671 - accuracy: 0.9519
Epoch 3/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1401 - accuracy: 0.9578
Epoch 4/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1324 - accuracy: 0.9599
Epoch 5/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1260 - accuracy: 0.9617
Epoch 6/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1226 - accuracy: 0.9630
Epoch 7/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1191 - accuracy: 0.9640
Epoch 8/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1159 - accuracy: 0.9647
Epoch 9/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1144 - accuracy: 0.9655
Epoch 10/10
2500/2500 [==============================] - 14s 5ms/step - l

In [92]:
test_loss = model.evaluate(x_test, y_test)
for name, value in zip(model.metrics_names, test_loss):
    print(name, value)


625/625 [==============================] - 3s 4ms/step - loss: 0.1091 - accuracy: 0.9680
loss 0.10908476263284683
accuracy 0.9679999947547913


In [103]:
# show every prediction of the model
predictions = model.predict(x_test)
predictions = predictions.argmax(axis=1)



In [108]:
# join y_test and predictions in a new dataframe
df_pred = pd.DataFrame(predictions)
df_pred = pd.merge(df_pred, y_test, left_index=True, right_index=True)
df_pred.columns = ['predictions', 'y_test']
df_pred['confirmation'] = df_pred.apply(lambda x: 'correct' if x['predictions'] == x['y_test'] else 'wrong', axis=1)
df_pred['confirmation'].value_counts()


wrong      2299
correct    1782
Name: confirmation, dtype: int64